<a href="https://colab.research.google.com/github/takayuki1997/fukushima/blob/master/Fukushima_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「福島県ものづくり企業データベース」から企業データを取り出すプログラム
http://www4.pref.fukushima.jp/maker/  
**福島県 商工労働部商工総務課**及び**公益財団福島県産業振興センター 経営支援グループ**が作成している福島県内のものづくり企業を集めたデータベースです。  
ところが、当者が異動してしまい保守がされていない。新しいデータの追加がされていない。データの取り出し方が分からない。  
手動で各企業データを表示させ、そこからコピペで企業名やメールアドレスを取得することは可能であるが、860社もあるので非常に大変。  
そこで、プログラムを使い自動的にウェブサイトにアクセスし、データを取得することにした。

## 企業データについて
データ数は860社（実際には859社だった）  
データ項目（25項目）
>業種
創業年
資本金
従業員数
沿革 
売上高
取得資格
代表者役職
代表者氏名
郵便番号
所在地
工業団地
URL
問合せ部署
担当者名
TEL
FAX
Eメールアドレス
本社所在地
本社資本金
本社従業員数 
国内事業所・工場等
海外事業所・工場等
企業名
ふりがな

## プログラムにつて
- pythonで書いた
- Google colaboratoryで走らせることを前提として書かれている。
- マウントしたgoogle driveに保存するようにした。
- ネットワークやサーバに負荷がかからないようにループごとに1秒待つ。
- 文字化けに気をつけた。（lxml，utf_8_sig）
- files.downloadは件数が多いとうまくいかない。
- 企業名とふりがなの位置を修正しようとするがうまくいかないのでそのままにしてある。

2020年4月3日  
荻 多加之


In [0]:
#@title
import request，
from bs4 import BeautifulSoup
import pandas as pd
import time
#from google.colab import files


# サーチするmakerID
company_number = range(0,5) # 1000
#company_number = range(3001,4001) # 1000

# 保存するファイルの名前
#save_file_name = 'Fukushima_data.csv'
save_file_name = '/content/drive/My Drive/Colab Notebooks/Fukushima_data.csv' # マウントしたgoogle drive
save_option = 1 # 0:not save, 1:save

#========================================================


print('Start ==========')

# 最初の１社だけ実行
m = 2
url = 'http://www4.pref.fukushima.jp/maker/search/detail.php?maker=' + str(m)
res = requests.get(url)
soup = BeautifulSoup(res.content,'lxml')
company_name = soup.find('h3')
company_name = company_name.text.strip()
kakko = company_name.rfind('(')
company_name1 = company_name[:kakko]
company_name2 = company_name[kakko+1:-1]
df = pd.read_html(url)
df = df[0]
df = df.set_index(0)
df = df.rename(columns={1:'initial'})
df.loc['企業名'] = company_name1
df.loc['ふりがな'] = company_name2


# forループ
for m in company_number:
  # 相手方のサーバやネットワークに負担をかけない様に1秒待ってみる
  time.sleep(1)
  #print(m)

  url = 'http://www4.pref.fukushima.jp/maker/search/detail.php?maker=' + str(m)

  # 企業名の取得
  res = requests.get(url)
  soup = BeautifulSoup(res.content,'lxml')
  company_name = soup.find('h3')
  if company_name is not None:
    company_name = company_name.text.strip()
    kakko = company_name.rfind('(')
    company_name1 = company_name[:kakko]
    company_name2 = company_name[kakko+1:-1]
    print('%5d %s' % (m, company_name1))

    # 企業情報の取得
    dfs = pd.read_html(url)
    dfs = dfs[0]
    dfs = dfs.set_index(0)
    dfs = dfs.rename(columns={1:m})

    # 企業情報と企業名を結合
    dfs.loc['企業名'] = company_name1
    dfs.loc['ふりがな'] = company_name2

    # 結合
    df = pd.concat([df, dfs], axis=1)


# initial削除
df = df.drop('initial', axis=1)

# 入れ替え
#display(df)
#df = df.loc[['企業名','ふりがな']]
#df = df.loc[-2:]
#display(df)

# 転置
df = df.T

# 表示
#display(df)

# csvとして保存 /content/
if save_option:
  df.to_csv(save_file_name, encoding='utf_8_sig')
  #files.download(save_file_name)
  print('Saved %s' % save_file_name)
else:
  print('Not saved')

# 終了
print('Fine========================')
